In [1]:
# Import findspark and initialize.
import findspark
findspark.init()
print("FindSpark initialized successfully!")

ValueError: Couldn't find Spark, make sure SPARK_HOME env is set or Spark is in an expected location (e.g. from homebrew installation).

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add the file to Spark's context
spark.sparkContext.addFile(url)

# Load the CSV file into a DataFrame
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the first few rows to verify the data
df.show(5)

In [ ]:
# 2. Create a temporary view of the DataFrame.
# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT YEAR(date) AS year_sold, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold
"""

# Execute the query
result_df = spark.sql(query)

# Show the results
result_df.show()


In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
SELECT YEAR(date) AS year_built, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year_built
ORDER BY year_built
"""

# Execute the query
result_df = spark.sql(query)

# Show the results
result_df.show()

In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
SELECT YEAR(date) AS year_built, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 
  AND bathrooms = 3 
  AND floors = 2 
  AND sqft_living >= 2000
GROUP BY year_built
ORDER BY year_built
"""

# Execute the query
result_df = spark.sql(query)

# Show the results
result_df.show()

In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
import time

# Start timer
start_time = time.time()
query = """
SELECT view, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.
# Cache the home_sales temporary table
spark.sql("CACHE TABLE home_sales")

# Confirm caching
print("Temporary table 'home_sales' has been cached.")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
query = """
SELECT view, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""
# Show results
result_df.show()


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Define the output path for the partitioned Parquet file
# Change output path to a directory inside your home directory
output_path = "/Users/andrewmeza/Desktop/Module_22/home_sales_partitioned"

# Save the DataFrame as a Parquet file partitioned by "date_built"
df.write.partitionBy("date_built").parquet(output_path, mode="overwrite")


# Save the DataFrame as a Parquet file partitioned by "date_built"
df.write.partitionBy("date_built").parquet(output_path, mode="overwrite")

print("Partitioned Parquet file has been successfully saved!")

In [ ]:
# 11. Read the formatted parquet data.
# Define the path where the partitioned Parquet file was saved
parquet_path = "/Users/andrewmeza/Desktop/Module_22/home_sales_partitioned"

# Read the partitioned Parquet data
df_parquet = spark.read.parquet(parquet_path)

# Show the first few rows to verify
df_parquet.show(5)

In [ ]:
# 12. Create a temporary table for the parquet data.
# Create a temporary SQL view
df_parquet.createOrReplaceTempView("home_sales_parquet")

print("Temporary table 'home_sales_parquet' has been created successfully!")

# Show available tables
spark.catalog.listTables()

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
# Run the query using the Parquet DataFrame
query = """
SELECT view, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales_parquet
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

# Execute the query
result_df = spark.sql(query)

# Show results
result_df.show()

# End timer and print execution time
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.isCached("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
print(f"Is 'home_sales' still cached? {is_cached}")